In [1]:
def create_sample_size_rec(data_src, data_labels, rejection_region, desired_power):
    
    import random
    import numpy as np
    import pandas as pd
    import statsmodels.api as sm
    
    data_src.columns = list(data_labels)
    
    absolute_mde = data_src[data_src['Treated'] == 1]['Order_Amt'].mean() - \
                   data_src[data_src['Treated'] == 0]['Order_Amt'].mean()

    
    print("The absolute MDE was estimated as {}.".format(absolute_mde))
    
    df = data_src[data_src['Treated'] == 0]
    assignment = []
    i = 0
    while i < len(df):
        assignment.append(random.randint(0,1)) 
        i += 1
    df['Partition'] = assignment
    power_analysis_df = df[df['Partition'] == 0]
    analysis_df = df[df['Partition'] == 1]
        
    del df
    
    pa_retailer_means = pd.DataFrame(power_analysis_df.groupby(['Retailer_ID'])['Order_Amt'].mean())
    pa_retailer_means.reset_index(inplace=True)
    pa_retailer_means.columns = ['Retailer_ID', 'Mean_Retailer_Order_Amt']
    ###############################################################################
    pa_dow_means = pd.DataFrame(power_analysis_df.groupby(['Dow_Rand'])['Order_Amt'].mean())
    pa_dow_means.reset_index(inplace=True)
    pa_dow_means.columns = ['Dow_Rand', 'Mean_DOW_Order_Amt']
    ###############################################################################
    analysis_df = pd.merge(analysis_df, pa_retailer_means, on='Retailer_ID', how='left')
    analysis_df = pd.merge(analysis_df, pa_dow_means, on='Dow_Rand', how='left')
    ###############################################################################
    analysis_df = analysis_df[['Order_Amt', 'Customer_ID', 'Mean_Order_Amt', 
                               'Mean_Retailer_Order_Amt','Mean_DOW_Order_Amt']]
    ###############################################################################
    analysis_df = analysis_df.dropna(how = 'any')
    
#     X = analysis_df[['Mean_Order_Amt', 'Mean_Retailer_Order_Amt','Mean_DOW_Order_Amt']]
#     X = analysis_df[['Mean_Retailer_Order_Amt','Mean_DOW_Order_Amt']]
    X = analysis_df[['Mean_DOW_Order_Amt']]
    X = sm.add_constant(X)
    Y = analysis_df[['Order_Amt']]
    residuals_df = sm.OLS(Y.astype(float), X.astype(float)).fit()
    
    X2 = analysis_df[['Customer_ID']]
    X2['Residual'] = residuals_df.resid
    X2['Constant'] = 1
    clustered_res = sm.OLS(X2['Residual'], X2['Constant']).fit(method='pinv'). \
                       get_robustcov_results('cluster', groups = X2['Customer_ID'], 
                       use_correction=True, df_correction=True)
    
    clustered_sd = clustered_res.bse[0] * np.sqrt(analysis_df.shape[0])
    effect_size = absolute_mde / clustered_sd
    recommended_n = int(sm.stats.tt_ind_solve_power(effect_size = effect_size, 
                        alpha = rejection_region, power = desired_power, 
                        alternative = 'larger'))
    print("A sample size of {} was recommended.".format(recommended_n ))
    return recommended_n, absolute_mde

In [2]:
def verify_sample_size_est(sample_size, data_src, data_labels, alpha, verify_n_times):

    import statsmodels.api as sm
    import pandas as pd
    
    SAMPLE_SIZE = sample_size
    VERIFICATION_ITERATIONS = verify_n_times
    ALPHA = alpha

    i = 0
    pvals  = []
    r_sqr  = []
    cond_n = []
    while i < VERIFICATION_ITERATIONS:
        working_df = data_src.sample(SAMPLE_SIZE, replace=False)
        ###############################################################################
        pa_retailer_means = pd.DataFrame(working_df.groupby(['Retailer_ID'])['Order_Amt'].mean())
        pa_retailer_means.reset_index(inplace=True)
        pa_retailer_means.columns = ['Retailer_ID', 'Mean_Retailer_Order_Amt']
        ###############################################################################
        pa_dow_means = pd.DataFrame(working_df.groupby(['Dow_Rand'])['Order_Amt'].mean())
        pa_dow_means.reset_index(inplace=True)
        pa_dow_means.columns = ['Dow_Rand', 'Mean_DOW_Order_Amt']
        ###############################################################################
        analysis_df = pd.merge(working_df, pa_retailer_means, on='Retailer_ID', how='left')
        analysis_df = pd.merge(analysis_df, pa_dow_means, on='Dow_Rand', how='left')
        ###############################################################################
        analysis_df = analysis_df[['Order_Amt', 'Customer_ID', 'Treated', 'Mean_Order_Amt', 
                                 'Mean_Retailer_Order_Amt','Mean_DOW_Order_Amt']]
        ###############################################################################
        analysis_df = analysis_df.dropna(how = 'any')
        
#         X = analysis_df[['Treated', 'Mean_Order_Amt', 'Mean_Retailer_Order_Amt','Mean_DOW_Order_Amt']]
        X = analysis_df[['Treated','Mean_DOW_Order_Amt']]
        X = sm.add_constant(X)
        Y = analysis_df[['Order_Amt']]
        model = sm.OLS(Y.astype(float), X.astype(float)).fit(method='pinv'). \
                       get_robustcov_results('cluster', groups = analysis_df['Customer_ID'], 
                       use_correction=True, df_correction=True)
        if model.pvalues[1] < ALPHA: 
            pvals.append(1)
        else:
            pvals.append(0)  
        r_sqr.append(model.rsquared_adj)
        cond_n.append(model.condition_number)
        i += 1
        if i % int((VERIFICATION_ITERATIONS)/10.0) == 0:
            completion = str(round((i/VERIFICATION_ITERATIONS)*100, 2))+'%'
            print(completion + ' complete.')
            
    # ----- Exit inner loop     
    x = ['Treated','Mean_DOW_Order_Amt']
    str_out = 'Order_Amt =' 
    d = 0
    for i in x:
        if d < 1:
            k = " '" + i + "'"
        else:
            k = " + '" + i + "'"
        str_out += k
        d += 1    
    
    actual_power = sum(pvals)/len(pvals)  
    mean_r_sqr   = sum(r_sqr)/len(r_sqr)   
    mean_cond_n  = sum(cond_n)/len(cond_n)  
    print("Actual power was estimated at {}.".format(actual_power))
    return actual_power, mean_r_sqr, mean_cond_n, str_out

In [3]:
def create_n_rec(pa_file_in, analysis_csv_in, headers_in, alpha, desired_power, point_verifications):
    import numpy as np
    import pandas as pd
    from scipy.optimize import curve_fit
    
    a_src = pd.read_csv(pa_file_in)
    b_src = pd.read_csv(analysis_csv_in)

    recommended_n, absolute_mde = create_sample_size_rec(a_src, headers_in, alpha, desired_power)

    if int(recommended_n*0.0005) < 10:
        pt_1 = 10    
    else:
        pt_1 = int(recommended_n*0.001)
    if int(recommended_n*0.005) < 10:
        pt_2 = 10    
    else:
        pt_2 = int(recommended_n*0.01)
    if int(recommended_n*0.05) < 10:
        pt_3 = 10    
    else:
        pt_3 = int(recommended_n*0.1)
    pt_4 = int(recommended_n)
    print("Proposed the following points [{},{},{},{}] for pass 1.".format(pt_1,pt_2,pt_3,pt_4))

    points = [pt_1, pt_2, pt_3, pt_4]

    actual_power_pass_1 = []
    for i in points:
        actual_power, mean_r_sqr, mean_cond_n, str_out = verify_sample_size_est(i, 
                                                                                b_src, 
                                                                                headers_in, 
                                                                                alpha, 
                                                                                point_verifications)
        actual_power_pass_1.append(actual_power)
        
    df = pd.DataFrame(points, actual_power_pass_1)
    df = df.reset_index(drop=False)
    df.columns = ['power', 'n']
    ub = df[df['power'] > 0.8]
    ub.sort_values('n', inplace=True, ascending=True)
    ub = ub.iat[0,1]
    lb = df[df['power'] < 0.8]
    lb.sort_values('n', inplace=True, ascending=False)
    lb = lb.iat[0,1]
    print('The upper-bound for pass 1 was found to be {}.'.format(ub))
    print('The lower-bound for pass 1 was found to be {}.'.format(lb))
    new_points = []
    for i in np.linspace(lb, ub, 5, endpoint = False):
        new_points.append(int(i))
    
    print("Proposed the following points [{},{},{},{},{}] for pass 2.".format(new_points[0],
                                                                              new_points[1],
                                                                              new_points[2],
                                                                              new_points[3],
                                                                              new_points[4]))
    actual_power_pass_2 = []
    for i in new_points:
        actual_power, mean_r_sqr, mean_cond_n, str_out = verify_sample_size_est(i, 
                                                                                b_src, 
                                                                                headers_in, 
                                                                                alpha, 
                                                                                point_verifications)
        actual_power_pass_2.append(actual_power)
        
    df = pd.DataFrame(new_points, actual_power_pass_2)
    df = df.reset_index(drop=False)
    df.columns = ['power', 'n']
    print(df)

    def exp_func(x, a, b, c):
        return a * np.log(b * x) + c
    desired_power = 0.8
    cdf = df['power']
    eta = df['n']
    popt, pcov = curve_fit(exp_func, eta, cdf)
    recommended_n = int(np.exp((desired_power - popt[2])/popt[0])/popt[1])
    return recommended_n

In [4]:
def meta_analysis(meta_n, power_n, final_verification_n, file_a, file_b, file_out):
    import time
    import pandas as pd

    dir = '/home/bknight/Documents/Power_Analysis_Techniques/variance_of_residuals/residual_dfs/'
    a_src    = dir + file_a
    b_src    = dir + file_b
    b_src_df = pd.read_csv(b_src)
    headers_in = ['Order_ID', 'Customer_ID', 'Mean_Order_Amt', 'Treated',
                  'Treatment_Modifier', 'Retailer_ID', 'Retailer_Scalar',
                  'Dow_Rand', 'DOW', 'Noise', 'Order_Amt']
    
    power_ls = []
    n_ls = []
    pass_1_p_ls = []
    pass_1_n_ls = []
    pass_2_p_ls = []
    pass_2_n_ls = []
    mean_r_sqr_ls = []
    mean_cond_n_ls = []
    str_out_ls = []
    verification_n_ls = [] 
    estimation_time_ls = []
    for i in list(range(0,meta_n,1)):
        start = time.time()
        n = create_n_rec(a_src, b_src, headers_in, 0.05, 0.8, power_n)
        print("A final value of {} was recommended.".format(n))
        end = time.time()
        delta = (end - start)/60.0
        print("The estimation took {} minutes.".format(delta))
        actual_power, mean_r_sqr, mean_cond_n, str_out = verify_sample_size_est(n, b_src_df, 
                                                         headers_in, 0.05, final_verification_n)
        print("The actual power attained was {}.".format(actual_power))
        
        power_ls.append(actual_power)
        n_ls.append(n)
        mean_r_sqr_ls.append(mean_r_sqr)
        mean_cond_n_ls.append(mean_cond_n)
        str_out_ls.append(str_out)
        pass_1_p_ls.append(4)
        pass_1_n_ls.append(power_n)
        pass_2_p_ls.append(5)
        pass_2_n_ls.append(power_n)
        verification_n_ls.append(final_verification_n)
        estimation_time_ls.append(delta)

    df_out = pd.DataFrame(
        {'Effectve Power': power_ls,
         'Recommended N': n_ls,
         'Mean R-Squared': mean_r_sqr_ls,
         'Mean Cond. No.': mean_cond_n_ls,
         'Specification': str_out_ls,
         'Initial Pass Points': pass_1_p_ls, 
         'Initial Pass Iterations per Point': pass_1_n_ls,
         'Second Pass Points': pass_2_p_ls,
         'Second Pass Iterations per Point': pass_2_n_ls,
         'Verification Iterations': verification_n_ls,
         'Estimation Time': estimation_time_ls
        })
    df_out.to_csv(file_out)
    return df_out

In [5]:
df = meta_analysis(100, 200, 500, 'part_II_df_mde_0_005_n_100000_a.csv', 
                                  'part_II_df_mde_0_005_n_100000_b.csv',
                                  'comb_methd_model_III_mde_0_005_n_100000.csv')
df

The absolute MDE was estimated as 0.42411496603367027.


/home/bknight/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/bknight/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/bknight/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/

A sample size of 467034 was recommended.
Proposed the following points [467,4670,46703,467034] for pass 1.
10.0% complete.
20.0% complete.
30.0% complete.
40.0% complete.
50.0% complete.
60.0% complete.
70.0% complete.
80.0% complete.
90.0% complete.
100.0% complete.
Actual power was estimated at 0.07.
10.0% complete.
20.0% complete.
30.0% complete.
40.0% complete.
50.0% complete.
60.0% complete.
70.0% complete.
80.0% complete.
90.0% complete.
100.0% complete.
Actual power was estimated at 0.105.
10.0% complete.
20.0% complete.
30.0% complete.
40.0% complete.
50.0% complete.
60.0% complete.
70.0% complete.
80.0% complete.
90.0% complete.
100.0% complete.
Actual power was estimated at 0.31.
10.0% complete.
20.0% complete.
30.0% complete.
40.0% complete.
50.0% complete.
60.0% complete.
70.0% complete.
80.0% complete.
90.0% complete.
100.0% complete.
Actual power was estimated at 0.905.
The upper-bound for pass 1 was found to be 467034.
The lower-bound for pass 1 was found to be 46703.
Pr

/home/bknight/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/bknight/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


10.0% complete.
20.0% complete.
30.0% complete.
40.0% complete.
50.0% complete.
60.0% complete.
70.0% complete.
80.0% complete.
90.0% complete.
100.0% complete.
Actual power was estimated at 0.28.
10.0% complete.
20.0% complete.
30.0% complete.
40.0% complete.
50.0% complete.
60.0% complete.
70.0% complete.
80.0% complete.
90.0% complete.
100.0% complete.
Actual power was estimated at 0.51.
10.0% complete.
20.0% complete.
30.0% complete.
40.0% complete.
50.0% complete.
60.0% complete.
70.0% complete.
80.0% complete.
90.0% complete.
100.0% complete.
Actual power was estimated at 0.695.
10.0% complete.
20.0% complete.
30.0% complete.
40.0% complete.
50.0% complete.
60.0% complete.
70.0% complete.
80.0% complete.
90.0% complete.
100.0% complete.
Actual power was estimated at 0.765.
10.0% complete.
20.0% complete.
30.0% complete.
40.0% complete.
50.0% complete.
60.0% complete.
70.0% complete.
80.0% complete.
90.0% complete.
100.0% complete.
Actual power was estimated at 0.85.
   power     

...

10.0% complete.
20.0% complete.
30.0% complete.
40.0% complete.
50.0% complete.
60.0% complete.
70.0% complete.
80.0% complete.
90.0% complete.
100.0% complete.
Actual power was estimated at 0.72.
10.0% complete.
20.0% complete.
30.0% complete.
40.0% complete.
50.0% complete.
60.0% complete.
70.0% complete.
80.0% complete.
90.0% complete.
100.0% complete.
Actual power was estimated at 0.855.
   power       n
0  0.345   45812
1  0.530  128273
2  0.700  210735
3  0.720  293197
4  0.855  375659
A final value of 350247 was recommended.
The estimation took 8.519503716627757 minutes.
10.0% complete.
20.0% complete.
30.0% complete.
40.0% complete.
50.0% complete.
60.0% complete.
70.0% complete.
80.0% complete.
90.0% complete.
100.0% complete.
Actual power was estimated at 0.802.
The actual power attained was 0.802.
The absolute MDE was estimated as 0.42411496603367027.
A sample size of 459683 was recommended.
Proposed the following points [459,4596,45968,459683] for pass 1.
10.0% complete.
20

10.0% complete.
20.0% complete.
30.0% complete.
40.0% complete.
50.0% complete.
60.0% complete.
70.0% complete.
80.0% complete.
90.0% complete.
100.0% complete.
Actual power was estimated at 0.806.
The actual power attained was 0.806.
The absolute MDE was estimated as 0.42411496603367027.
A sample size of 460280 was recommended.
Proposed the following points [460,4602,46028,460280] for pass 1.
10.0% complete.
20.0% complete.
30.0% complete.
40.0% complete.
50.0% complete.
60.0% complete.
70.0% complete.
80.0% complete.
90.0% complete.
100.0% complete.
Actual power was estimated at 0.025.
10.0% complete.
20.0% complete.
30.0% complete.
40.0% complete.
50.0% complete.
60.0% complete.
70.0% complete.
80.0% complete.
90.0% complete.
100.0% complete.
Actual power was estimated at 0.09.
10.0% complete.
20.0% complete.
30.0% complete.
40.0% complete.
50.0% complete.
60.0% complete.
70.0% complete.
80.0% complete.
90.0% complete.
100.0% complete.
Actual power was estimated at 0.295.
10.0% comp

10.0% complete.
20.0% complete.
30.0% complete.
40.0% complete.
50.0% complete.
60.0% complete.
70.0% complete.
80.0% complete.
90.0% complete.
100.0% complete.
Actual power was estimated at 0.105.
10.0% complete.
20.0% complete.
30.0% complete.
40.0% complete.
50.0% complete.
60.0% complete.
70.0% complete.
80.0% complete.
90.0% complete.
100.0% complete.
Actual power was estimated at 0.365.
10.0% complete.
20.0% complete.
30.0% complete.
40.0% complete.
50.0% complete.
60.0% complete.
70.0% complete.
80.0% complete.
90.0% complete.
100.0% complete.
Actual power was estimated at 0.925.
The upper-bound for pass 1 was found to be 462331.
The lower-bound for pass 1 was found to be 46233.
Proposed the following points [46233,129452,212672,295891,379111] for pass 2.
10.0% complete.
20.0% complete.
30.0% complete.
40.0% complete.
50.0% complete.
60.0% complete.
70.0% complete.
80.0% complete.
90.0% complete.
100.0% complete.
Actual power was estimated at 0.265.
10.0% complete.
20.0% complet

10.0% complete.
20.0% complete.
30.0% complete.
40.0% complete.
50.0% complete.
60.0% complete.
70.0% complete.
80.0% complete.
90.0% complete.
100.0% complete.
Actual power was estimated at 0.27.
10.0% complete.
20.0% complete.
30.0% complete.
40.0% complete.
50.0% complete.
60.0% complete.
70.0% complete.
80.0% complete.
90.0% complete.
100.0% complete.
Actual power was estimated at 0.52.
10.0% complete.
20.0% complete.
30.0% complete.
40.0% complete.
50.0% complete.
60.0% complete.
70.0% complete.
80.0% complete.
90.0% complete.
100.0% complete.
Actual power was estimated at 0.675.
10.0% complete.
20.0% complete.
30.0% complete.
40.0% complete.
50.0% complete.
60.0% complete.
70.0% complete.
80.0% complete.
90.0% complete.
100.0% complete.
Actual power was estimated at 0.775.
10.0% complete.
20.0% complete.
30.0% complete.
40.0% complete.
50.0% complete.
60.0% complete.
70.0% complete.
80.0% complete.
90.0% complete.
100.0% complete.
Actual power was estimated at 0.82.
   power     

,Effectve Power,Recommended N,Mean R-Squared,Mean Cond. No.,Specification,Initial Pass Points,Initial Pass Iterations per Point,Second Pass Points,Second Pass Iterations per Point,Verification Iterations,Estimation Time
0,0.834,332985,0.362278,814.360533,Order_Amt = 'Treated' + 'Mean_DOW_Order_Amt',4,200,5,200,500,9.684886
1,0.788,312168,0.362192,814.421891,Order_Amt = 'Treated' + 'Mean_DOW_Order_Amt',4,200,5,200,500,10.196452
2,0.812,320834,0.362240,814.410970,Order_Amt = 'Treated' + 'Mean_DOW_Order_Amt',4,200,5,200,500,10.333212
3,0.852,354137,0.362280,814.322034,Order_Amt = 'Treated' + 'Mean_DOW_Order_Amt',4,200,5,200,500,9.550878
4,0.784,338434,0.362289,814.418030,Order_Amt = 'Treated' + 'Mean_DOW_Order_Amt',4,200,5,200,500,9.963245
5,0.816,338980,0.362251,814.422093,Order_Amt = 'Treated' + 'Mean_DOW_Order_Amt',4,200,5,200,500,9.522834
6,0.832,330832,0.362255,814.381639,Order_Amt = 'Treated' + 'Mean_DOW_Order_Amt',4,200,5,200,500,9.253030
7,0.832,379025,0.362240,814.464663,Order_Amt = 'Treated' + 'Mean_DOW_Order_Amt',4,200,5,200,500,9.309929
8,0.830,348076,0.362249,814.416847,Order_Amt = 'Treated' + 'Mean_DOW_Order_Amt',4,200,5,200,500,9.474608
9,0.780,308204,0.362356,814.225680,Order_Amt = 'Treated' + 'Mean_DOW_Order_Amt',4,200,5,200,500,10.193529
